In [13]:
import sys
import re
import os
# Twitter API
import tweepy
from tweepy import OAuthHandler
# Reddit API
import praw
# Text Analyzing
from textblob import TextBlob
# MongoDB
from pymongo import MongoClient
import json

# stockName = "AAPL"
stockName = sys.argv[1]

In [ ]:

class SentimentAnalyzer(object):
    '''
    Generic Twitter Class for sentiment analysis
    '''

    def __init__(self):
        '''
        Class constructor method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = "9LanGta2OOzJBITyWRacIyxUR"
        consumer_secret = "mAfvIGHwHblw0bMJfszyn2voADCUWLEAT1ezbgPPiZb1geHi8X"
        access_token = "1428093851726663696-55Xp7zjUQknkuPRDzCObGCcIQuxmOh"
        access_token_secret = "1jxsg4RdkszPMURIRi78aNm2hoHwMzz4VrBU69gmOV0gm"

        # keys and id from the Reddit Dev Console
        

        # connection to MongoDB
        self.client = MongoClient('localhost', 27017)
        self.db = self.client.numeraxial
        self.collections = self.db.list_collection_names()
        self.collection_name = self.db.sentiments
        
        # attempt authentication
        try:

            # TWITTER
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)

            # REDDIT
            self.reddit = praw.Reddit(client_id = "18uKEpIOKA-Q9egZ9cqluA",
                                    client_secret = "Wsg75jOgphdmZ64JZcbKWqbzoXemQg",
                                    user_agent = "Scraping")
            self.all = self.reddit.subreddit("WallStreetBets+stocks+investing+pennystocks+robinhood")

        except:
            print("Error: Authentication Failed")

    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def get_tweets(self, query, count=10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []

        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q=query, count=count)

            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}

                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(
                    tweet.text)

                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            ptweets = [
                tweet for tweet in tweets if tweet['sentiment'] == 'positive']
            ntweets = [
                tweet for tweet in tweets if tweet['sentiment'] == 'negative']
            percentagePTweets = 100*len(ptweets)/len(tweets)
            percentageNTweets = 100*len(ntweets)/len(tweets)
            percentageNUTweets = 100 * \
                (len(tweets) - (len(ntweets)+len(ptweets)))/len(tweets)
            
            return {
                'Positive': percentagePTweets,
                'Negative': percentageNTweets,
                'Neutral': percentageNUTweets
            }

        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

    def get_reddit(self, query):
        sentiment_analysis = {'positive': 0,
                              'negative': 0,
                              'neutral': 0
                              }

        for submission in self.all.search(query,"Stock", limit=None):
            analysis = TextBlob(submission.title)
            if analysis.sentiment.polarity > 0:
                sentiment_analysis['positive'] += 1
            elif analysis.sentiment.polarity == 0:
                sentiment_analysis['neutral'] += 1
            else:
                sentiment_analysis['negative'] += 1

        total = sum(sentiment_analysis.values())/100
        percentagePReddit = float('{:.2f}'.format(
            sentiment_analysis['positive']/total))
        percentageNReddit = float('{:.2f}'.format(
            sentiment_analysis['negative']/total))
        percentageNUReddit = float('{:.2f}'.format(
            sentiment_analysis['neutral']/total))

        return {
            'Positive': percentagePReddit,
            'Negative': percentageNReddit,
            'Neutral': percentageNUReddit
        }

    def get_sentiment(self, query, count=10):
        twitter_result = self.get_tweets(query, count)
        reddit_result = self.get_reddit(query)
        
        # Hash result
        twitterPositive = twitter_result['Positive']
        entry = {
            'Stock': query,
            'Sentiment': {
                'Twitter': {
                    'Positive': twitter_result['Positive'],
                    'Negative': twitter_result['Negative'],
                    'Neutral': twitter_result['Neutral'],
                },
                'Reddit': {
                    'Positive': reddit_result['Positive'],
                    'Negative': reddit_result['Negative'],
                    'Neutral': reddit_result['Neutral'],
                }
            }

        }
        self.collection_name.update_one(
            {'Stock': query}, {"$set": entry}, True)



def main():
    # creating object of TwitterClient Class
    api = SentimentAnalyzer()
    # calling function to get tweets
    api.get_sentiment(query="Macy's", count=200)


main()

